In [3]:
import torch
import numpy as np
from misc import data, c
from torch import optim
from scipy.stats import spearmanr
from vae2 import VAE # import the last version

def get_cor_ensamble(batch, mutants_values, model, ensambles = 512, rand = True):
    model.eval()

    mt_elbos, wt_elbos = 0, 0

    for i in range(ensambles):
        if i and (i % 2 == 0):
            print(f"\tReached {i}/rand={rand}", " "*32, end="\r")

        elbos     = model.logp(batch, rand=rand).detach().cpu()
        wt_elbos += elbos[0]
        mt_elbos += elbos[1:]

    print()

    diffs  = (mt_elbos / ensambles) - (wt_elbos / ensambles)
    cor, _ = spearmanr(mutants_values, diffs)
    
    return cor

ModuleNotFoundError: No module named 'torch'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataloader, df, mutants_tensor, mutants_df, neff = data(batch_size = 64, device=device)

wildtype   = dataloader.dataset[0] # one-hot-encoded wildtype 
eval_batch = torch.cat([wildtype.unsqueeze(0), mutants_tensor.to(device)])

args = {
    'alphabet_len': dataloader.dataset[0].shape[0],
    'seq_len':      dataloader.dataset[0].shape[1],
    'neff':         neff
}

vae   = VAE(**args).to(device)
opt   = optim.Adam(vae.parameters())

stats = {
    'rl': [],  # rl  = Reconstruction loss
    'klz': [], # kl  = Kullback-Leibler divergence loss
    'klp': [],  # KL divergence loss for parameters
    'cor': []  # cor = Spearman correlation to experimentally measured 
    }          # protein fitness according to eq.1 from paper

Parsing fasta 'data/BLAT_ECOLX_hmmerbit_plmc_n5_m30_f50_t0.2_r24-286_id100_b105.a2m'
Parsing labels 'data/BLAT_ECOLX_hmmerbit_plmc_n5_m30_f50_t0.2_r24-286_id100_b105_LABELS.a2m'
Generating 8403 1-hot encodings
Generating 8403 1-hot encodings. Took 0.793s torch.Size([8403, 23, 253])
Generating 4807 1-hot encodings
Generating 4807 1-hot encodings. Took 0.421s torch.Size([4807, 23, 253])


In [ ]:
vae

VAE(
  (fc1): Linear(in_features=5819, out_features=64, bias=True)
  (fc1h): Linear(in_features=64, out_features=30, bias=True)
  (fc21): Linear(in_features=30, out_features=2, bias=True)
  (fc22): Linear(in_features=30, out_features=2, bias=True)
  (fc3): Linear(in_features=2, out_features=32, bias=False)
  (fc3h): Linear(in_features=32, out_features=64, bias=False)
  (W): Linear(in_features=16, out_features=16192, bias=False)
  (C): Linear(in_features=23, out_features=16, bias=False)
  (S): Linear(in_features=253, out_features=8, bias=False)
)

In [ ]:
for epoch in range(30):
    # Unsupervised training on the MSA sequences.
    # https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch
    vae.train()
    
    epoch_losses = { 'rl': [], 'klp': [], 'klz': [] }
    for batch in dataloader:
        # https://discuss.pytorch.org/t/what-step-backward-and-zero-grad-do/33301/2
        opt.zero_grad()
        x_hat, mu, logvar  = vae(batch)
        loss, rl, klz, klp = vae.loss(x_hat, batch, mu, logvar)
        loss.mean().backward(retain_graph=True) # Stefan: Do we need 'retain_graph'? - ask yevgen 
        opt.step()
        epoch_losses['rl'].append(rl.mean().item())
        epoch_losses['klp'].append(klp.mean().item())
        epoch_losses['klz'].append(klz.item())

    # Evaluation on mutants
    vae.eval()
    # x_hat_eval, mu, logvar = vae(eval_batch, rep=False)
    # elbos, _, _, _ = vae.loss(x_hat_eval, eval_batch, mu, logvar)
    # elbos = vae.logp(eval_batch)
    # diffs       = elbos[1:] - elbos[0] # log-ratio (first equation in the paper)
    # cor, _      = spearmanr(mutants_df.value, diffs.detach().to('cpu'))
    cor = get_cor_ensamble(eval_batch, mutants_df.value, vae, ensambles=16, rand=True)
    
    # Populate statistics 
    stats['rl'].append(np.mean(epoch_losses['rl']))
    stats['klz'].append(np.mean(epoch_losses['klz']))
    stats['klp'].append(np.mean(epoch_losses['klp']))
    stats['cor'].append(np.abs(cor))

    to_print = [
        f"{c.HEADER}EPOCH %03d"          % epoch,
        f"{c.OKBLUE}RL=%4.4f"            % stats['rl'][-1], # reconstrution loss
        f"{c.OKGREEN}KLZ=%4.4f"          % stats['klz'][-1], # KL loss
        f"{c.OKCYAN}|rho|=%4.4f{c.ENDC}" % stats['cor'][-1] # correlation (we want to max this value)
    ]
    print(" ".join(to_print))

	Reached 14/rand=True                                 
EPOCH 000 RL=778.7021 KLZ=0.1985 |rho|=0.4288

EPOCH 001 RL=748.3698 KLZ=0.0257 |rho|=0.4620

EPOCH 002 RL=712.2460 KLZ=0.0307 |rho|=0.5206

EPOCH 003 RL=670.7138 KLZ=0.0251 |rho|=0.5455

EPOCH 004 RL=628.1255 KLZ=0.0167 |rho|=0.5637

EPOCH 005 RL=589.5910 KLZ=0.0115 |rho|=0.5766

EPOCH 006 RL=558.5648 KLZ=0.0081 |rho|=0.5817

EPOCH 007 RL=538.0320 KLZ=0.0061 |rho|=0.5864

EPOCH 008 RL=527.0403 KLZ=0.0043 |rho|=0.5911

EPOCH 009 RL=519.9695 KLZ=0.0040 |rho|=0.5948

EPOCH 010 RL=514.7305 KLZ=0.0039 |rho|=0.5973

EPOCH 011 RL=511.7554 KLZ=0.0042 |rho|=0.5998

EPOCH 012 RL=512.0990 KLZ=0.0040 |rho|=0.6011

EPOCH 013 RL=509.3880 KLZ=0.0047 |rho|=0.6022

EPOCH 014 RL=507.8127 KLZ=0.0044 |rho|=0.6027

EPOCH 015 RL=507.8654 KLZ=0.0049 |rho|=0.6019

EPOCH 016 RL=508.3556 KLZ=0.0048 |rho|=0.6021

EPOCH 017 RL=507.3471 KLZ=0.0052 |rho|=0.6021

EPOCH 018 RL=506.6234 KLZ=0.0049 |rho|=0.6028

EPOCH 019 RL=507.4076 KLZ=0.0055 |rho|=0.6021

EPOCH

In [ ]:
torch.save({
    'state_dict': vae.state_dict(), 
    'stats':      stats,
    'args':       args,
}, "models/full_paper.model.pth")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=35838f82-2ce6-4453-9bd2-2d87a43af151' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>